<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name: </b> deeplearn8 <br>
<b> Student1: </b> Claudio SCALZO <br>
<b> Student2: </b> Luca LOMBARDO <br>
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
# SUPPRESS WARNINGS
import warnings
warnings.filterwarnings("ignore")


# IMPORT LIBRARIES
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


# LOAD DATA (RESHAPING THE IMAGES)
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
X_train, y_train = mnist.train.images.reshape((-1,28,28,1)), mnist.train.labels
X_val, y_val     = mnist.validation.images.reshape((-1,28,28,1)), mnist.validation.labels
X_test, y_test   = mnist.test.images.reshape((-1,28,28,1)), mnist.test.labels


# PRINT SHAPES
print("Image Shape:    {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_val)))
print("Test Set:       {} samples".format(len(X_test)))


# DECLARE NOISE
epsilon = 1e-10

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape:    (28, 28, 1)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


# Section 1 : Neural Network in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [2]:
#STEP 1

# Parameters
learning_rate = 0.01
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 1.288760053
Epoch:  02   =====> Loss= 0.732404479
Epoch:  03   =====> Loss= 0.600406613
Epoch:  04   =====> Loss= 0.536792513
Epoch:  05   =====> Loss= 0.498032909
Epoch:  06   =====> Loss= 0.471232529
Epoch:  07   =====> Loss= 0.451284567
Epoch:  08   =====> Loss= 0.436038171
Epoch:  09   =====> Loss= 0.423383205
Epoch:  10   =====> Loss= 0.413319309
Epoch:  11   =====> Loss= 0.404056309
Epoch:  12   =====> Loss= 0.397012284
Epoch:  13   =====> Loss= 0.390438593
Epoch:  14   =====> Loss= 0.384509605
Epoch:  15   =====> Loss= 0.379378294
Epoch:  16   =====> Loss= 0.374423921
Epoch:  17   =====> Loss= 0.370338481
Epoch:  18   =====> Loss= 0.366582157
Epoch:  19   =====> Loss= 0.362834374
Epoch:  20   =====> Loss= 0.359873165
Epoch:  21   =====> Loss= 0.356709092
Epoch:  22   =====> Loss= 0.353878512
Epoch:  23   =====> Loss= 0.351120375
Epoch:  24   =====> Loss= 0.348726849
Epoch:  25   =====> Loss= 0.346465973
Epoch:  26   =====> Loss= 0.344176202
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge!

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html

##### <img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


<b> Question 2.1.1 </b>  Implement the Neural Network architecture described above.
For that, your will use classes and functions from  https://www.tensorflow.org/api_docs/python/tf/nn. 

We give you some helper functions for weigths and bias initilization. Also you can refer to section 1. 


In [3]:
# --- FUNCTIONS FOR WEIGHT AND BIAS INITIALIZATIONS --- #
# WEIGHTS
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

# BIASES
def bias_variable(shape):
  initial = tf.constant(0., shape=shape)
  return tf.Variable(initial)

In [4]:
def LeNet5_Model(images):    
    
    # --- INTERNAL MODEL DEFINITION --- #
    with tf.name_scope("model"):
        
        # CONVOLUTIONAL 1 LAYER
        with tf.name_scope("conv1"):
            
            # Convolution (28x28x1 -> CONV1 -> 28x28x6)
            conv1 = tf.nn.conv2d(input=images,
                                 filter=weight_variable([5,5,1,6]),
                                 strides=[1,1,1,1],
                                 padding='SAME')
            
            # Activation
            act1 = tf.nn.relu(conv1 + bias_variable([6]))
            
            # Pooling (28x28x6 -> MAXPOOL1 -> 14x14x6)
            pool1 = tf.nn.max_pool(act1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    
    
        # CONVOLUTIONAL 2 LAYER
        with tf.name_scope('conv2'):
            
            # Convolution (14x14x6 -> CONV2 -> 10x10x16)
            conv2 = tf.nn.conv2d(input=pool1,
                                 filter=weight_variable([5,5,6,16]),
                                 strides=[1,1,1,1],
                                 padding='VALID')
            
            # Activation
            act2 = tf.nn.relu(conv2 + bias_variable([16]))
            
            # Pooling (10x10x16 -> MAXPOOL2 -> 5x5x16)
            pool2 = tf.nn.max_pool(act2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')   
        
        
        # FLATTEN LAYER
        with tf.name_scope('flatten'):
            
            # Flatten (5x5x16 -> 400)
            flat = tf.contrib.layers.flatten(pool2)
        
        
        # FULLY CONNECTED LAYER 3
        with tf.name_scope('layer3'):
            
            # Activation (400 -> FC3 -> 120)
            hidden3 = tf.nn.relu(tf.matmul(flat, weight_variable([400,120])) + bias_variable([120]))
            
            
        # FULLY CONNECTED LAYER 4
        with tf.name_scope('layer4'):
            
            # Activation (120 -> FC4 -> 84)
            hidden4 = tf.nn.relu(tf.matmul(hidden3, weight_variable([120,84])) + bias_variable([84]))
            
            
        # FULLY CONNECTED LAYER 5
        with tf.name_scope('layer5'):
            
            # Activation (84 -> FC5 -> 10)
            predictions = tf.nn.softmax(tf.matmul(hidden4, weight_variable([84,10])) + bias_variable([10]))
        
        
        return predictions

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

In [15]:
# COMPUTE THE NUMBER OF PARAMETERS
count = np.sum([np.prod(v.shape.as_list()) for v in tf.trainable_variables()])

# PRINT THE RESULT
print("Total number of parameters:", count)

Total number of parameters: 61706


<div class='alert alert-success'>
The number of parameters is <b>61706</b>*. This number has been computed thanks to the <tt>tf.trainable_variables()</tt> function available in TensorFlow:<br>
<br>
- For each variable, the shape is computed.<br>
- The components of the shape are multiplied, so the number of parameters regarding that shape is obtained.<br>
- All the parameters regarding each shape are then summed together.<br>
<br>
&#42;<i>note: this cell, even if it's located before, has been executed <b>after</b> the following cell, to be run with the right instance of the model (the CNN), and not with the one of the classic neural network.<i>
</div>

<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Function: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [17]:
# RESET THE GRAPH AND CLEAR LOGS
! rm -rf ./log_files
tf.reset_default_graph()


# PARAMETERS
learning_rate = 0.001
training_epochs = 40
batch_size = 128
loss_display_step = 1
acc_display_step = 10
logs_path = 'log_files/'


# PLACEHOLDERS
x = tf.placeholder(tf.float32, [None,28,28,1], name='input')
y = tf.placeholder(tf.float32, [None,10], name='output')


# MODEL DEFINITION
# Model structure
with tf.name_scope('model'):
    pred = LeNet5_Model(images=x)
    
# Loss
with tf.name_scope('loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))

# Optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

# Accuracy
with tf.name_scope('accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [18]:
# EVALUATION FUNCTION
def evaluate(logits, labels):
    
    correct = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    return tf.reduce_mean(tf.cast(correct, tf.float32))

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 100 epochs)
- Save the model after training
- Print after training the final testing accuracy 

In [19]:
# TRAIN FUNCTION
def train(init, sess, logs_path, n_epochs, batch_size, optimizer, cost, merged_summary_op):
    
    # Start
    print("[OPTIMIZATION STARTED]\n")
    sess.run(init)
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # Training
    for epoch in range(training_epochs):
        
        # Init
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        
        # Loop over all batches
        for i in range(total_batch):
            
            # Next batch
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            batch_xs = batch_xs.reshape(batch_size,28,28,1)
            
            # Run optimization
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            
            # Write logs
            summary_writer.add_summary(summary, epoch * total_batch + i)
            
            # Compute average loss
            avg_cost += c / total_batch
        
        # Display logs
        if (epoch+1) % loss_display_step == 0:
            print("Epoch:", '%02d' % (epoch+1), " =====>  Loss =", "{:.4f}".format(avg_cost))
        
        if (epoch+1) % acc_display_step == 0:
            acc_train = acc.eval({x: X_train, y: y_train})
            acc_val = acc.eval({x: X_val, y: y_val})
            
            print("Train accuracy =", "{:.2f}%".format(acc_train*100),
                  " | ",
                  "Validation accuracy =", "{:.2f}%\n".format(acc_val*100))
            
            
    print("[OPTIMIZATION FINISHED]")
    summary_writer.flush()
    
    
    # --- ACCURACY ------------------------------------------------------ #
    acc_test = acc.eval({x: X_test, y: y_test})
    print("Test accuracy =", "{:.2f}%".format(acc_test*100))

<div class='label label-success'>LeNet architecture (SGD optimizer)</div>

In [20]:
# MAKE SUMMARIES
init = tf.global_variables_initializer()
tf.summary.scalar("loss-LeNet", cost)
tf.summary.scalar("accuracy-LeNet", acc)
merged_summary_op = tf.summary.merge_all()


# LAUNCH SESSION
with tf.Session() as sess:
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

[OPTIMIZATION STARTED]

Epoch: 01  =====>  Loss = 2.2890
Epoch: 02  =====>  Loss = 2.2523
Epoch: 03  =====>  Loss = 2.1899
Epoch: 04  =====>  Loss = 2.0412
Epoch: 05  =====>  Loss = 1.6634
Epoch: 06  =====>  Loss = 1.1185
Epoch: 07  =====>  Loss = 0.7754
Epoch: 08  =====>  Loss = 0.6134
Epoch: 09  =====>  Loss = 0.5278
Epoch: 10  =====>  Loss = 0.4735
Train accuracy = 86.95%  |  Validation accuracy = 87.46%

Epoch: 11  =====>  Loss = 0.4345
Epoch: 12  =====>  Loss = 0.4046
Epoch: 13  =====>  Loss = 0.3807
Epoch: 14  =====>  Loss = 0.3614
Epoch: 15  =====>  Loss = 0.3455
Epoch: 16  =====>  Loss = 0.3313
Epoch: 17  =====>  Loss = 0.3194
Epoch: 18  =====>  Loss = 0.3088
Epoch: 19  =====>  Loss = 0.2990
Epoch: 20  =====>  Loss = 0.2902
Train accuracy = 91.43%  |  Validation accuracy = 91.98%

Epoch: 21  =====>  Loss = 0.2823
Epoch: 22  =====>  Loss = 0.2746
Epoch: 23  =====>  Loss = 0.2671
Epoch: 24  =====>  Loss = 0.2608
Epoch: 25  =====>  Loss = 0.2546
Epoch: 26  =====>  Loss = 0.2491
Ep

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Graph:
<img src="./MNIST_figures/graph.png" width=600px><br>

<img src="./MNIST_figures/loss.png" width=1000px><br>

<img src="./MNIST_figures/accuracy.png" width=1000px>

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |        94.91%      |       98.77%        |       
| Training Time        |         ???        |        ???          |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** ...


<div class='label label-success'>LeNet architecture (Adam optimizer)</div>

In [9]:
# RESET THE GRAPH AND CLEAR LOGS
! rm -rf ./log_files
tf.reset_default_graph()


# PARAMETERS
learning_rate = 0.001
training_epochs = 40
batch_size = 128
loss_display_step = 1
acc_display_step = 10
logs_path = 'log_files/'


# PLACEHOLDERS
x = tf.placeholder(tf.float32, [None,28,28,1], name='input')
y = tf.placeholder(tf.float32, [None,10], name='output')


# MODEL DEFINITION
# Model structure
with tf.name_scope('model'):
    pred = LeNet5_Model(images=x)
    
# Loss
with tf.name_scope('loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))

# Optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Accuracy
with tf.name_scope('accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    

# MAKE SUMMARIES
init = tf.global_variables_initializer()
tf.summary.scalar("loss-LeNet", cost)
tf.summary.scalar("accuracy-LeNet", acc)
merged_summary_op = tf.summary.merge_all()


# LAUNCH SESSION
with tf.Session() as sess:
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

[OPTIMIZATION STARTED]

Epoch: 01  =====>  Loss = 0.3346
Epoch: 02  =====>  Loss = 0.0931
Epoch: 03  =====>  Loss = 0.0653
Epoch: 04  =====>  Loss = 0.0500
Epoch: 05  =====>  Loss = 0.0407
Epoch: 06  =====>  Loss = 0.0327
Epoch: 07  =====>  Loss = 0.0276
Epoch: 08  =====>  Loss = 0.0237
Epoch: 09  =====>  Loss = 0.0223
Epoch: 10  =====>  Loss = 0.0185
Train accuracy = 99.38%  |  Validation accuracy = 98.76%

Epoch: 11  =====>  Loss = 0.0180
Epoch: 12  =====>  Loss = 0.0165
Epoch: 13  =====>  Loss = 0.0133
Epoch: 14  =====>  Loss = 0.0135
Epoch: 15  =====>  Loss = 0.0103
Epoch: 16  =====>  Loss = 0.0103
Epoch: 17  =====>  Loss = 0.0112
Epoch: 18  =====>  Loss = 0.0092
Epoch: 19  =====>  Loss = 0.0084
Epoch: 20  =====>  Loss = 0.0077
Train accuracy = 99.76%  |  Validation accuracy = 99.02%

Epoch: 21  =====>  Loss = 0.0072
Epoch: 22  =====>  Loss = 0.0053
Epoch: 23  =====>  Loss = 0.0068
Epoch: 24  =====>  Loss = 0.0059
Epoch: 25  =====>  Loss = 0.0033
Epoch: 26  =====>  Loss = 0.0115
Ep

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** ...

In [13]:
def LeNet5_Model_Dropout(images):
    
    # --- INTERNAL MODEL DEFINITION --- #
    with tf.name_scope("model"):
        
        # CONVOLUTIONAL 1 LAYER
        with tf.name_scope("conv1"):

            # Convolution (28x28x1 -> CONV1 -> 28x28x6)
            conv1 = tf.nn.conv2d(input=images,
                                 filter=weight_variable([5,5,1,6]),
                                 strides=[1,1,1,1],
                                 padding='SAME')

            # Activation
            act1 = tf.nn.relu(conv1 + bias_variable([6]))

            # Pooling (28x28x6 -> MAXPOOL1 -> 14x14x6)
            pool1 = tf.nn.max_pool(act1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')


        # CONVOLUTIONAL 2 LAYER
        with tf.name_scope('conv2'):

            # Convolution (14x14x6 -> CONV2 -> 10x10x16)
            conv2 = tf.nn.conv2d(input=pool1,
                                 filter=weight_variable([5,5,6,16]),
                                 strides=[1,1,1,1],
                                 padding='VALID')

            # Activation
            act2 = tf.nn.relu(conv2 + bias_variable([16]))

            # Pooling (10x10x16 -> MAXPOOL2 -> 5x5x16)
            pool2 = tf.nn.max_pool(act2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')   


        # FLATTEN LAYER
        with tf.name_scope('flatten'):

            # Flatten (5x5x16 -> 400)
            flat = tf.contrib.layers.flatten(pool2)


        # DROPOUT LAYER
        with tf.name_scope('dropout'):

            # Dropout (0.75)
            flat_drop = tf.nn.dropout(flat, keep_prob = 0.75)


        # FULLY CONNECTED LAYER 3
        with tf.name_scope('layer3'):

            # Activation (400 -> FC3 -> 120)
            hidden3 = tf.nn.relu(tf.matmul(flat_drop, weight_variable([400,120])) + bias_variable([120]))


        # FULLY CONNECTED LAYER 4
        with tf.name_scope('layer4'):

            # Activation (120 -> FC4 -> 84)
            hidden4 = tf.nn.relu(tf.matmul(hidden3, weight_variable([120,84])) + bias_variable([84]))


        # FULLY CONNECTED LAYER 5
        with tf.name_scope('layer5'):

            # Activation (84 -> FC5 -> 10)
            predictions = tf.nn.softmax(tf.matmul(hidden4, weight_variable([84,10])) + bias_variable([10]))


        return predictions

<div class='label label-success'>LeNet architecture with Dropout (Adam optimizer)</div>

In [14]:
# RESET THE GRAPH AND CLEAR LOGS
! rm -rf ./log_files
tf.reset_default_graph()


# PARAMETERS
learning_rate = 0.001
training_epochs = 40
batch_size = 128
loss_display_step = 1
acc_display_step = 10
logs_path = 'log_files/'


# PLACEHOLDERS
x = tf.placeholder(tf.float32, [None,28,28,1], name='input')
y = tf.placeholder(tf.float32, [None,10], name='output')


# MODEL DEFINITION
# Model structure (with dropout)
with tf.name_scope('model'):
    pred = LeNet5_Model_Dropout(images=x)
    
# Loss
with tf.name_scope('loss'):
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))

# Optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Accuracy
with tf.name_scope('accuracy'):
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))
    

# MAKE SUMMARIES
init = tf.global_variables_initializer()
tf.summary.scalar("loss-LeNet", cost)
tf.summary.scalar("accuracy-LeNet", acc)
merged_summary_op = tf.summary.merge_all()


# LAUNCH SESSION
with tf.Session() as sess:
    train(init, sess, logs_path, training_epochs, batch_size, optimizer, cost, merged_summary_op)

[OPTIMIZATION STARTED]

Epoch: 01  =====>  Loss = 0.3814
Epoch: 02  =====>  Loss = 0.1187
Epoch: 03  =====>  Loss = 0.0874
Epoch: 04  =====>  Loss = 0.0713
Epoch: 05  =====>  Loss = 0.0622
Epoch: 06  =====>  Loss = 0.0543
Epoch: 07  =====>  Loss = 0.0483
Epoch: 08  =====>  Loss = 0.0445
Epoch: 09  =====>  Loss = 0.0396
Epoch: 10  =====>  Loss = 0.0367
Train accuracy = 98.90%  |  Validation accuracy = 98.44%

Epoch: 11  =====>  Loss = 0.0367
Epoch: 12  =====>  Loss = 0.0340
Epoch: 13  =====>  Loss = 0.0309
Epoch: 14  =====>  Loss = 0.0293
Epoch: 15  =====>  Loss = 0.0276
Epoch: 16  =====>  Loss = 0.0248
Epoch: 17  =====>  Loss = 0.0248
Epoch: 18  =====>  Loss = 0.0237
Epoch: 19  =====>  Loss = 0.0227
Epoch: 20  =====>  Loss = 0.0215
Train accuracy = 99.18%  |  Validation accuracy = 98.90%

Epoch: 21  =====>  Loss = 0.0217
Epoch: 22  =====>  Loss = 0.0191
Epoch: 23  =====>  Loss = 0.0189
Epoch: 24  =====>  Loss = 0.0191
Epoch: 25  =====>  Loss = 0.0185
Epoch: 26  =====>  Loss = 0.0170
Ep